In [2]:
import os
import numpy as np
import xarray as xr
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# output_dir = "/Users/danywaller/Projects/mercury/test10_2025dec10/"
output_dir = "/Users/danywaller/Projects/mercury/extreme/CPS_Base/"

# -------------------------
# Load data
# -------------------------
# ds = xr.open_dataset("/Users/danywaller/Projects/mercury/test10_2025dec10/out/Amitis_field_100000.nc")
ds = xr.open_dataset("/Users/danywaller/Projects/mercury/extreme/CPS_Base/object/Amitis_CPS_Base_115000_xz_comp.nc")
t = 0  # snapshot

# Normalized coordinates (R_m units)
x_all = ds["Nx"].values / 2440
y_all = ds["Ny"].values / 2440
z_all = ds["Nz"].values / 2440

# J components on (time, Nz, Ny, Nx)
Jx_all = ds["Jx"].isel(time=t).values
Jy_all = ds["Jy"].isel(time=t).values
Jz_all = ds["Jz"].isel(time=t).values

# --------------------------------
# Build index masks
# --------------------------------
ix = np.where((x_all > -3) & (x_all < 3))[0]
iy = np.where((y_all > -3) & (y_all < 3))[0]
iz = np.where((z_all > -3) & (z_all < 3))[0]

# --------------------------------
# Subset coordinates
# --------------------------------
x = x_all[ix]
y = y_all[iy]
z = z_all[iz]

# --------------------------------
# Subset J components
# Order is (Nz, Ny, Nx)
# --------------------------------
Jx = Jx_all[np.ix_(iz, iy, ix)]
Jy = Jy_all[np.ix_(iz, iy, ix)]
Jz = Jz_all[np.ix_(iz, iy, ix)]

# ----------------------------
# Compute Jmag and |grad Jmag|
# ----------------------------
Jmag = np.sqrt(Jx**2 + Jy**2 + Jz**2)

# Use coordinate-aware spacing. np.gradient expects spacing per axis matching array order.
# Array order is (Nz, Ny, Nx) -> spacing is (z, y, x)
dJdz, dJdy, dJdx = np.gradient(Jmag, z, y, x, edge_order=1)
gradJ = np.sqrt(dJdx**2 + dJdy**2 + dJdz**2)

# ----------------------------
# Subsample first (performance)
# ----------------------------
step = 2  # increase if heavy
x_s = x[::step]
y_s = y[::step]
z_s = z[::step]

Jx_s = Jx[::step, ::step, ::step]
Jy_s = Jy[::step, ::step, ::step]
Jz_s = Jz[::step, ::step, ::step]
gradJ_s = gradJ[::step, ::step, ::step]

Zg, Yg, Xg = np.meshgrid(z_s, y_s, x_s, indexing="ij")

# ----------------------------
# Threshold: keep only strong grad regions
# ----------------------------
pct = 90
thr = np.nanpercentile(gradJ_s, pct)

mask = np.isfinite(gradJ_s) & (gradJ_s >= thr)
inds = np.argwhere(mask)

if len(inds) == 0:
    raise ValueError("No points exceeded threshold; lower pct or reduce step.")

# Downsample the selected points so Plotly stays responsive
max_arrows = 4000
if len(inds) > max_arrows:
    # stratified-ish selection by sorting on grad strength
    flat_grad = gradJ_s[mask]
    order = np.argsort(flat_grad)[::-1]  # strongest first
    inds = inds[order[:max_arrows]]

ii, jj, kk = inds[:, 0], inds[:, 1], inds[:, 2]

# Positions
Xp = Xg[ii, jj, kk]
Yp = Yg[ii, jj, kk]
Zp = Zg[ii, jj, kk]

# Vectors (J)
Up = Jx_s[ii, jj, kk]
Vp = Jy_s[ii, jj, kk]
Wp = Jz_s[ii, jj, kk]

# Optional: normalize arrow lengths so direction is emphasized
# (comment out if you want magnitude-scaled arrows)
Jnorm = np.sqrt(Up**2 + Vp**2 + Wp**2)
Jnorm[Jnorm == 0] = np.nan
Upn, Vpn, Wpn = Up / Jnorm, Vp / Jnorm, Wp / Jnorm

# grad values at selected points
G = gradJ_s[ii, jj, kk].astype(float)

# Normalize / clip gradient for stable visual scaling
Gmax = np.nanmax(G)
if not np.isfinite(Gmax) or Gmax == 0:
    raise ValueError("Gradient values are all zero/NaN after thresholding.")

g = G / Gmax                      # 0..1
g = np.clip(g, 0, 1)

# Direction = J-hat (already computed as Upn,Vpn,Wpn)
# Magnitude encodes gradient strength so that cone colors track |∇Jmag|
Uplot = Upn * g
Vplot = Vpn * g
Wplot = Wpn * g

fig = go.Figure()

fig.add_trace(go.Cone(
    x=Xp, y=Yp, z=Zp,
    u=Uplot, v=Vplot, w=Wplot,
    colorscale="Viridis",
    cmin=0.0, cmax=1.0,
    showscale=True,
    colorbar=dict(title="|J| (normalized)"),
    sizemode="scaled",
    sizeref=1.2,          # tune for your coordinate units (try 0.5–3)
    anchor="tail",
    customdata=np.stack([G], axis=1),
))

# -------------------------------
# PLANET SPHERE
# -------------------------------
u = np.linspace(0, 2*np.pi, 60)
v = np.linspace(0, np.pi, 30)
Xs = (1) * np.outer(np.cos(u), np.sin(v))
Ys = (1) * np.outer(np.sin(u), np.sin(v))
Zs = (1) * np.outer(np.ones_like(u), np.cos(v))

sphere = go.Surface(x=Xs, y=Ys, z=Zs, colorscale=[[0,"lightgray"],[1,"lightgray"]],
                    opacity=0.75, showscale=False)

fig.add_trace(sphere)

fig.update_layout(
    title=f"J vectors where |J| > {pct/100}*max(|J|)",
    scene=dict(
        xaxis=dict(range=[-3, 3], title="X (Rₘ)"), 
        yaxis=dict(range=[-3, 3], title="Y (Rₘ)"), 
        zaxis=dict(range=[-3, 3], title="Z (Rₘ)"), 
        aspectmode="cube"),
    margin=dict(l=50, r=50, t=50, b=50),
    width=1000,
    height=800,
)

out_html = "J_arrows_highJmag.html"
fig.write_html(os.path.join(output_dir, out_html), include_plotlyjs="cdn")
# print(f"Wrote {out_html}")
fig.show()
